## DD Utilities

In [ ]:
import numpy as np

In [ ]:
def Linear_system(n,sread,swrite,iparam):
    if sread == False:
        M = n*np.identity(n) + np.random.rand(n,n) #uses uniform dist on [0,1)
        y= np.random.rand(n,1)
    else:
        from pandas import read_csv
        df = read_csv('iparam'+str(iparam)+'.csv',header=None)
        print("didn't fully read")
#         from pandas import read_csv
#         df = read_csv('M.csv',header=None)
#         M = df.values
#         df = read_csv('y.csv',header=None)
#         y = df.values
        pass
    
    x= np.linalg.solve(M,y)
    xstore= x
    
    return M, x, y, xstore

In [ ]:
def save_to_csv(n,p,K,nb,nn,ne,M,y,svpert,iparam):
    import csv
    f = open('iparam'+str(iparam)+'.csv', 'w')
    writer = csv.writer(f)
    writer.writerow([n])
    writer.writerow([p])
    writer.writerow([K])
    writer.writerow(nb)
    writer.writerow(nn)
    writer.writerow(ne)
    writer.writerows(M)
    writer.writerow(y[:,0])
    f.close()
    
    file = open('svpert'+str(iparam)+'.csv', 'w')
    writer = csv.writer(file)
    writer.writerows(svpert)
    file.close()

In [ ]:
def set_parameters(n,p,b,e):
    #checks for inconsistent input, throws errors
    #establishes p, b, and e if needed
    import math
    pgiven=p!=-1
    bgiven=b!=[-1]
    egiven=e!=[-1]
    overlapmode=type(e)==int # I didn't do any typechecking, so it won't throw an error for a float
  
    if n<2 or n>100:
         raise ValueError("error n should be between 2 and 100")
        
    if pgiven and (p<2 or p>50):
        raise ValueError("error p should be between 2 and 50")
    
    if bgiven:
        if p != len(b):#b must be a list
            if not pgiven: 
                p = len(b)
            else:
                raise ValueError("error inconsistent input of b and p")

        if egiven:
            if overlapmode:
                if e<1:
                    raise ValueError("error overlap (integer given for e) should not be less than 1")
            else:
                if len(e) != len(b): # already verified that p is len b
                    raise ValueError("error inconsistent input of b and e")
        
        if (not egiven) or overlapmode:
            o=2
            if overlapmode:
                o=e

#             o=3
            e=[]
            for i in range(len(b)-1):
                e+=[b[i+1]+o-1]
                if e[-1]>n: #will reset any of them to be <= n
                    e[-1]=n
            e+=[n] #because the last end must be n
        if not verify_domains1(b,e,n):
            raise ValueError("error: set of subdomains does not give full coverage of the domain")

#         if not verify_domains2(b,e,n):
#             raise ValueError("error: Domains do not overlap sufficiently. If we represent the subdomains that overlap as a graph, there must only be once connected component. Now, there are more than 1.")


      # construct e
    else:
        o=3
        if egiven:
            if overlapmode:
                if e<1:
                    raise ValueError("error overlap (integer given for e) should not be less than 1")
                o=e
            else:
                raise ValueError("error please give b if you give a set of ends as e")
        if not pgiven:
            p=4 #since nothing was given
            
        # construct b and e since neither was given as a list
        inc=(n-1)/p
        val=1
        b=[0]*p
        for i in range(p):
            b[i]=math.ceil(val)
            val+=inc
        e=[]
        for i in range(len(b)-1):
            e+=[b[i+1]+o-1]
            if e[-1]>n: #will reset any of them to be <= n
                e[-1]=n
      #assert(b[i+1]+o-1<=n)#will fail if o>n-1
        e+=[n]
    
    return p,b,e

In [ ]:
def fix_domains(n, p, b, e, iprint):
    #creates nn, ne, nb, and index which are needed later
    
    import math
    
    #these should only fail if fix_domains failed
#     assert(verify_domains1(b,e,n)) 
#     assert(verify_domains2(b,e,n)) #should fail if o<1

    nb=np.array(b)
    ne=np.array(e)
    nn=ne-nb+1
    nnmax=np.max(nn)

    index=np.zeros((p,nnmax))
#     index[0,0:nn[0]]=np.linspace(nb[0],ne[0],nn[0])
#     index[1,0:nn[1]]=np.linspace(nb[1],ne[1],nn[1])
#     index[2,0:nn[2]]=np.linspace(nb[2],ne[2],nn[2])
#     index[3,0:nn[3]]=np.linspace(nb[3],ne[3],nn[3])
    for i in range(p):
        index[i,0:nn[i]]=np.linspace(nb[i],ne[i],nn[i])

    return nn, nb, ne, index

In [ ]:
def verify_domains1(b,e,n):
    # Determines whether the set of subdomains gives full coverage of the domain.
    # verify_domains1 is unnecessary because passing verify_domains2 implies that a set of domains 
    # passes verify_domains1.
    vec=[0]*n
    p=len(b)
    for i in range(p):
        if b[i]>e[i]:
            return False
    for i in range(p):
        for j in range(b[i],e[i]+1):
            vec[j-1]=1
    for s in vec:
        if s==0:
            return False
    return True

In [ ]:
def verify_domains2(b,e,n,verbose=False):
    # The subomains should overlap sufficiently. There should not be subgroups of subdomains that overlap, 
    # but do not overlap the other group.  If we represent the subdomains as vertices in a graph where an 
    # edge represents overlap, there must only be once connected component.
    
    p=len(b)
    M=np.zeros((n,n))
    vec=np.zeros((n,1))
    vec[-1]=1
    for i in range(p):
        for j in range(b[i],e[i]+1):
            for l in range(b[i],e[i]+1):
                M[j-1,l-1]=1
    if verbose:
        print(M)
        print(vec)
    M=np.linalg.matrix_power(M,n)#probably more efficient to associate from the right side
    vec=M@vec
    if verbose:
        print(M)
        print(vec)
    for s in vec:
        if s==0:
            return False
    return True

In [ ]:
def Rmatrices(n,p,nn,index,iprint):
    nnmax=np.max(nn)
    R=np.zeros((p,nnmax,n))
    
    for i in range(0,p):
        for j in range(0,nn[i]):
            R[i,j,int(index[i,j] - 1)] = 1 
    
    return R

In [ ]:
def Amatrices(M,R,n,p,nn,iprint):
    nnmax=np.max(nn)
    A=np.zeros((p,nnmax,nnmax))
    RR=np.zeros((nnmax,n))
    AA=np.zeros((nnmax,nnmax))
    
    for i in range(0,p):
        RR=R[i] 
        AA=RR@M@RR.T
        A[i]=AA
        
    return A

In [ ]:
def Bmatrices(A, R, n, p, nn, iprint):
    nnmax= max(nn)

    B= np.zeros((p,n,n))
    RR= np.zeros((nnmax,n))
    AA= np.zeros((nnmax,nnmax))
    
    for i in range(0,p):
        RR=R[i]
        AA=A[i]
        B[i]=RR[0:nn[i],:].T@np.linalg.inv(AA[0:nn[i],0:nn[i]])@RR[0:nn[i],:]
    
    if iprint >=6:
        print('Print B matrices')
        for i in range(p):
              tmp= np.zeros((n,n))
              print('i=',i)
              tmp[0:n,0:n]= B[i,0:n,0:n]
              tmp[1:n,1:n]
        
    return B

In [ ]:
def Cmatrices(M,B,n,p,iprint):
    C= np.zeros((p,n,n))
    CC= np.zeros((n,n))
    BB= np.zeros((n,n))
    for i in range(p):
        BB= B[i]
        CC= BB@M
        C[i]= CC

    if iprint >= 6:
        print('Print C matrices ')
        for i in range(p):
            tmp= np.zeros((n,n))
            print('Subdomain',i)
            tmp[1:n,1:n]= C[i,1:n,1:n]
            print(tmp[1:n,1:n])

    return C

In [ ]:
def fvector(B,y,n,p,iprint):
    # Construct f vector for either additive or multiplicative Schwarz
    
    f=np.zeros((p,n))
    BB=np.zeros((n,n))
    
    for i in range(p):
        BB=B[i]
        f[i]=BB@y[0:n,0]
    if iprint >= 4:
        print("Print f vector ")
        for i in range (p):
            tmp = np.zeros((n,1))
            print("i=",i)
            tmp[0:n,0] = f[i,0:n]
            print(tmp[0:n,0])
            
    return f

In [ ]:
def solution_compare(xstore,vstore,wstore,zstore,iprint):

    nx= len(xstore)
    nv= len(vstore)

    print(f'norm(exact - iterative) solutions    = {np.linalg.norm(xstore[0:nx,0]-vstore[nv-nx:nv,0]):13.6e}' ) 

    if iprint >= 2:
        print(f'Difference between vstore and wstore = {np.linalg.norm(vstore-wstore):13.6e}')
        print(f'Difference between wstore and zstore = {np.linalg.norm(wstore-zstore):13.6e}')
        print(f'Difference between vstore and zstore = {np.linalg.norm(vstore-zstore):13.6e}')

    if iprint >=6:
        print(xstore[0:nx,0]-vstore[nv-nx:nv,0])
        print(vstore-wstore)
        print(wstore-zstore)
        print(vstore-zstore)